Another key tool in developing data-intensive applications in python is descriptors.
Even though they are the heart of python's attribute access mechanism,  many seasonal developers are not aware of them.

Every python developer uses them without knowing about them. Few of the core functionalities in python which uses descriptors under the hood are:
1- Properties
2- Static and Class methods
3- Super
4- lru-cache in functools
5- Slots for declaring static attributes in class.
6- Field types in ORMs such as sqlalchemy.


Descriptors allow developer to customize the behavior of attribute retrieval, access, and deletion. Many of you must be thinking is that we can achieve the same using properties but additional benefit of descriptors is that you can reuse them to assign same behavior to multiple attributes in different objects.

## Use-cases of descriptors 
1. **Validation and Transformation:** Ensure attributes meet specific criteria during assignment.

2. **Lazy Loading:** Defer loading resource-intensive data until the attribute is accessed.

3. **Attribute Management:** Centrally control and customize behavior for multiple attributes.

4. **Atomic Operations with Descriptor Factories:** Dynamically generate descriptors with shared behavior.

5. **Attribute-Scoped Validation:** Apply attribute-specific constraints to enhance data integrity.

6. **Effortless Memoization:** Optimize performance by caching results of expensive computations.

7. **Customizing Attribute Retrieval:** Tailor the way attribute values are retrieved.

8. **Enforcing Constraints on Attribute Values:** Ensure attributes adhere to defined rules.

9. **Centralized Attribute Management:** Implement consistent behavior for attributes across classes.

10. **Facilitating Lazy Computation:** Delay computation until attribute access, improving efficiency.

11. **Implementing Read-Only Attributes:** Restrict attributes to be read-only.

12. **Enabling Calculated Attributes:** Compute attribute values on-the-fly based on other attributes.

13. **Dynamic Generation of Attributes:** Create attributes dynamically with shared functionality.

14. **Ensuring Consistent Data Types:** Enforce uniform data types for attributes.

15. **Intercepting Attribute Assignments:** Perform actions or checks when attributes are assigned.

16. **Implementing Attribute-specific Logic:** Define custom logic for individual attributes.

17. **Fine-grained Control over Attribute Access:** Customize access patterns for precise control.

18. **Optimizing Performance through Caching:** Improve speed by storing and reusing computed values.

19. **Implementing Data Descriptor for ORM Fields:** Use descriptors to define attributes in Object-Relational Mapping (ORM).

20. **Customizing Behavior of Class Methods:** Extend or modify behavior of class methods using descriptors.

21. **Implementing Attribute-level Security Checks:** Enforce security measures at the attribute level.

22. **Achieving Atomicity in Attribute Assignments:** Ensure consistency by performing atomic operations during assignment.

23. **Implementing Customizable Default Values:** Set default values with flexibility for customization.

24. **Dynamic Adjustment of Attribute Values:** Modify attribute values dynamically based on conditions.

25. **Implementing Reference Counting Mechanisms:** Manage references and control object lifecycle through attribute counting.


# How to define descriptors

Defining a descriptor in Python involves creating a class with one or more of the special methods __get__, __set__, and __delete__. These methods determine how attribute retrieval, assignment, and deletion are handled when an instance of the descriptor class is part of another object. 

Descriptor is a protocol-based feature. No subclassing is needed to implement it.

In [1]:
class MyDescriptor:
    def __get__(self, instance, owner):
        # Define behavior for attribute retrieval
        print(f"Getting attribute in {owner} instance")
        return instance._value

    def __set__(self, instance, value):
        # Define behavior for attribute assignment
        print(f"Setting attribute in {instance} instance to {value}")
        instance._value = value

    def __delete__(self, instance):
        # Define behavior for attribute deletion
        print(f"Deleting attribute in {instance} instance")
        del instance._value

class MyClass:
    descriptor_attribute = MyDescriptor()

    def __init__(self, initial_value):
        self._value = initial_value


Example usage

In [2]:
obj = MyClass(initial_value=42)
print(obj.descriptor_attribute)  # Invokes __get__
obj.descriptor_attribute = 99    # Invokes __set__
del obj.descriptor_attribute      # Invokes __delete__

Getting attribute in <class '__main__.MyClass'> instance
42
Setting attribute in <__main__.MyClass object at 0x1037868f0> instance to 99
Deleting attribute in <__main__.MyClass object at 0x1037868f0> instance


In above example, three arguments are used in `__get__`, `__set__` and `__delete__`; `self`, `instance` and `owner`. self is instance of descriptor class itself, `instance` is a instance of class (managed class) where descriptor is initialized and `owner` is reference to class of managed class. `owner` is use full in case we want to retrieve class attributes.



### Detailed Example

#### Validations and Transformations

We can use `__set__` method to validate input values before assigning them to field.

In below example, ValidationDescriptor is validating that input values are within range defined by validator.

In [7]:
class ValidationDescriptor:
    def __init__(self, min_=None, max_=None):
        self.default_value = 0
        self.min = min_
        self.max = max_

    def __set_name__(self, owner, name):
        self.name = name

    def __get__(self, instance, owner):
        print(f"Getting attribute in {owner} instance")
        return instance.__dict__.get(self.name, self.default_value)

    def __set__(self, instance, value):
        print(f"Setting attribute in {instance} instance to {value}")
        if not self.min <= value <= self.max:
            raise ValueError(f"{value} must be between {self.min} and {self.max}")
        instance.__dict__[self.name] = value


# Usage of ValidationDescriptor
class MyClass:
    value1 = ValidationDescriptor(min_=0, max_=100)
    value2 = ValidationDescriptor(min_=0, max_=100)
    def __init__(self, value1, value2):
        self.value1 = value1
        self.value2 = value2
        
obj = MyClass(40, 65)
print(obj.value1)  # Invokes __get__
print(obj.value2) # Invokes __get__
obj.int_value1 = 50    # Invokes __set__
print(obj.value1)  # Invokes __get__
print(obj.value2) # Invokes __get__

Setting attribute in <__main__.MyClass object at 0x1076da2c0> instance to 40
Setting attribute in <__main__.MyClass object at 0x1076da2c0> instance to 65
Getting attribute in <class '__main__.MyClass'> instance
40
Getting attribute in <class '__main__.MyClass'> instance
65
Getting attribute in <class '__main__.MyClass'> instance
40
Getting attribute in <class '__main__.MyClass'> instance
65


<div style="color:red">
    Note: Don't use getattr in `__get__` and setattr in `__set__` as it will put your code in infinite loop.
</div>